In [206]:
import json
import re

In [296]:
dict_file = './dictionary.json'
parse_file = './Lexical/Test06.txt'
output_file = './result.pasc'

In [297]:
dic = json.load(open(dict_file))
raw_text = open(parse_file).read()

In [298]:
raw_text = raw_text.replace('\n', ' \n')
print(repr(raw_text))

'x = y + 56'


In [299]:
isComment = 0
errors = []
results = []

In [300]:
def tokenParser(token, startRow, startCol, endRow, endCol):
    if isComment == 1:
        return

    if token == '':
        return
    
    if token in dic:
        results.append([token, dic[token] + 'number', startRow, endRow, startCol + 1, endCol + 1])
        return
    else:        
        if re.match(r'^[0-9]+$', token):
            results.append([token, 'ICONSTnumber', startRow, endRow, startCol + 1, endCol + 1])
            return
        
        if re.match(r'^\'.\'$', token):
            results.append([token, 'CCONSTnumber', startRow, endRow, startCol + 1, endCol + 1])
            return
        
        if re.match('^\'.*\'$', token):
            results.append([token, 'SCONSTnumber', startRow, endRow, startCol + 1, endCol + 1])
            return
                
        if re.match(r'^[0-9]+[a-zA-Z0-9]*$', token):
            errors.append([startRow, endRow, startCol + 1, endCol + 1, token, '[SYNTAX ERROR]: Can not use this this as an identifier'])
            return
        else:
            results.append([token, 'IDnumber', startRow, endRow, startCol + 1, endCol + 1])
            return
#         print(previous_token)
#     results.append([token, startRow, endRow, startCol + 1, endCol + 1])

In [301]:
rowCnt = 1
token = ''
idx = 0
startCol = 0
lastCol = 0
results = []
errors = []
spaceTab = 0

while idx < len(raw_text):
    ch = raw_text[idx]
#     print(repr(ch))
    if ch == '\n':
        rowCnt += 1
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        token = ''
        idx += 1
        spaceTab = 0
        startCol = idx
        lastCol = idx
        continue
    
    if ch == '\t':
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        token = ''
        idx += 1
        spaceTab += 3
        startCol = idx
        continue
        
    if ch == ' ':
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        token = ''
        idx += 1
        startCol = idx
        continue
    
    if ch == '(':
        if raw_text[idx + 1] == '*':
            isComment = 1
            token = ''
            idx += 1
            startCol = idx
            continue
        else:
            tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
            tokenParser(ch, rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
            token = ''
            idx += 1
            startCol = idx
            continue
        
    if ch == ')':
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        tokenParser(ch, rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
        token = ''
        idx += 1
        startCol = idx
        continue
    
    if ch == ',':
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        tokenParser(',', rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
        idx += 1
        startCol = idx
        token = ''
        continue
        
    if ch == '.':
        if raw_text[idx + 1] == '.':
            tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
            tokenParser('..', rowCnt, idx - lastCol + spaceTab, rowCnt, idx + 1 - lastCol + spaceTab)
            idx += 2
            token = ''
            startCol = idx
            continue
            
    if ch == ':':
        if raw_text[idx + 1] == '=':
            tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
            tokenParser(':=', rowCnt, idx - lastCol + spaceTab, rowCnt, idx + 1 - lastCol + spaceTab)
            idx += 2
            token = ''
            startCol = idx
            continue
        else:
            tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
            tokenParser(':', rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
            idx += 1
            token = ''
            startCol = idx
            continue
            
    if ch == ';':
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        tokenParser(ch, rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
        idx += 1
        startCol = idx
        token = ''
        continue
        
    if ch == '+':
        tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
        tokenParser(ch, rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
        idx += 1
        startCol = idx
        token = ''
        continue
        
    if ch == '-':
        if re.match(r'^[0-9]$', raw_text[idx + 1]):
            idx += 1
            token = token + ch
            continue
        else:
            tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
            tokenParser(ch, rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
            idx += 1
            startCol = idx
            token = ''
            continue
            
    if ch == '*':
        if raw_text[idx + 1] == ')' and isComment == 0:
            errors.append([rowCnt, rowCnt, idx - lastCol + spaceTab, idx - lastCol + spaceTab, '[ERROR]: Is this the end of comment?'])
            idx += 2
            continue
        elif raw_text[idx + 1] == ')':
            isComment = 0
            idx += 1
            startCol = idx
            token = ''
            continue
        else:
            tokenParser(token, rowCnt, startCol - lastCol + spaceTab, rowCnt, idx - 1 - lastCol + spaceTab)
            tokenParser(ch, rowCnt, idx - lastCol + spaceTab, rowCnt, idx - lastCol + spaceTab)
            idx += 1
            startCol = idx
            token = ''
            continue
    
    token = token + ch
    idx += 1

if len(token) > 0:
    tokenParser(token, rowCnt, startCol - lastCol, rowCnt, idx - 1 - lastCol)
    
print(rowCnt)

1


In [302]:
print('Parsing results:')
for i in results:
    print(i)

print('Errors list:')
for i in errors:
    print(i)
    
fileOut = open('out.txt', 'w+')
for i in results:
    fileOut.write(str(i[0]) + ' ' + (i[1]) + ' ' + str(i[2]) + ' ' + str(i[3]) + ' ' + str(i[4]) + ' ' + str(i[5]) + '\n')
fileOut.close()

Parsing results:
['x', 'IDnumber', 1, 1, 1, 1]
['=', 'EQnumber', 1, 1, 3, 3]
['y', 'IDnumber', 1, 1, 5, 5]
['+', 'PLUSnumber', 1, 1, 7, 7]
['56', 'ICONSTnumber', 1, 1, 9, 10]
Errors list:
